In [27]:
#import defencies
%matplotlib notebook
import tweepy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from config import (consumer_key, consumer_secret, access_token,access_token_secret)

#apikey
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

#vadersentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

#csv converting file
dataframe = "distinguishing_sentiments.csv"

#targetting users
target_users = ['@BBC','@CBS','@CNN','@FoxNews','@nytimes']

In [28]:

#pages (since its 20 per page)
page = 5

#making blank string for vadersentiment 
sentiments = []
compound_list = []
positive_list = []
negative_list = []
neutral_list = []
counter = 0

for target in target_users:
    
    for x in range(page):
        
        public_tweets = api.user_timeline(target, page=page)
        
        for tweet in public_tweets:
            results = analyzer.polarity_scores(tweet["text"])
        
            compound = results["compound"]
            pos = results["pos"]
            neu = results["neu"]
            neg = results["neg"]
            
            timestamped = tweet["created_at"]
            twit = tweet["user"]["screen_name"]
            text = tweet["text"]
            
            compound_list.append(compound)
            positive_list.append(pos)
            negative_list.append(neg)
            neutral_list.append(neu)
            
            #Storing average sentiments
            sentiments.append({
                "User": target,
                "Text": text,
                "Time of Tweet": timestamped,
                "Compound": np.mean(compound_list),
                "Positive": np.mean(positive_list),
                "Neutral": np.mean(negative_list),
                "Negative": np.mean(neutral_list),
                "Tweet Count": len(compound_list),
                "Tweets Ago": counter
            })
            
            
            counter += 1            

In [29]:
print(sentiments)

[{'User': '@BBC', 'Text': "🐻🍁This year's #Autumnwatch will take the team to New England, bringing you the most spectacular of seasons in four… https://t.co/lFI4SoY2yL", 'Time of Tweet': 'Wed Sep 12 09:02:01 +0000 2018', 'Compound': 0.0, 'Positive': 0.0, 'Neutral': 0.0, 'Negative': 1.0, 'Tweet Count': 1, 'Tweets Ago': 0}, {'User': '@BBC', 'Text': '🍔"My son\'s getting clean protein… and no cholesterol.”\n\nMeet the dad who feeds his son burgers almost every day.… https://t.co/OCCcPs1EDy', 'Time of Tweet': 'Wed Sep 12 08:01:07 +0000 2018', 'Compound': 0.064, 'Positive': 0.059, 'Neutral': 0.048, 'Negative': 0.893, 'Tweet Count': 2, 'Tweets Ago': 1}, {'User': '@BBC', 'Text': 'Every year 640,000 tonnes of plastic from the fishing industry enters the sea. 🎣\n\n♻️But there’s nowhere in the UK i… https://t.co/V8Hcc4L02V', 'Time of Tweet': 'Wed Sep 12 07:31:00 +0000 2018', 'Compound': 0.042666666666666665, 'Positive': 0.03933333333333333, 'Neutral': 0.032, 'Negative': 0.9286666666666666, 'Tweet 

In [32]:
sentiments_pd = pd.DataFrame.from_dict(sentiments)
sentiments_pd.head(2)

,Compound,Negative,Neutral,Positive,Text,Time of Tweet,Tweet Count,Tweets Ago,User
0,0.000,1.000,0.000,0.000,🐻🍁This year's #Autumnwatch will take the team ...,Wed Sep 12 09:02:01 +0000 2018,1,0,@BBC
1,0.064,0.893,0.048,0.059,"🍔""My son's getting clean protein… and no chole...",Wed Sep 12 08:01:07 +0000 2018,2,1,@BBC


In [31]:
#EXTRACTING DF TO CSV
print(len(sentiments_pd))
sentiments_pd.to_csv(dataframe)

500
